In [1]:
# import statements
import datetime as dt
import matplotlib.pyplot as plt
from matplotlib import style
import pandas as pd
import pandas_datareader.data as web
style.use('ggplot')

import requests

# truncate settings
pd.options.display.max_columns = None
pd.options.display.max_rows = None
pd.options.display.max_colwidth = None


ModuleNotFoundError: No module named 'pandas_datareader'

In [30]:
start = dt.datetime(2010,1,1)
end = dt.datetime(2021,12,31)

ticker = 'TSLA'
API_KEY = 'UNZXVIOI4TVWK6XZ'

In [31]:
# df = web.DataReader('TSLA', 'yahoo', start, end)
df = web.DataReader(ticker, 'av-daily', start, end, api_key='UNZXVIOI4TVWK6XZ')
df.to_csv('tsla.csv')

In [32]:
df = pd.read_csv('tsla.csv', parse_dates=True, index_col=0)

In [33]:
df.tail()

,open,high,low,close,volume
2021-12-27,1073.6700,1117.0000,1070.7152,1093.94,23715273
2021-12-28,1109.4900,1118.9999,1078.4200,1088.47,20107969
2021-12-29,1098.6400,1104.0000,1064.1400,1086.19,18718015
2021-12-30,1061.3300,1095.5500,1053.1500,1070.34,15680313
2021-12-31,1073.4444,1081.9999,1054.5900,1056.78,13466216


In [36]:
# list of tickers

tickers = ['AAPL', 'TLSA', 'BABA', 'FB', 'SPOT']

In [38]:
def get_financial_statement(ticker, statement):

  url = f'https://www.alphavantage.co/query?function={statement}&symbol={ticker}&apikey={API_KEY}'
  print(url)

  r = requests.get(url)

  data = r.json() # this returns a dict
  
  pnl = pd.DataFrame(data['annualReports'])

  # pnl['fiscalDateEnding'] = pd.to_datetime(pnl['fiscalDateEnding']) # convert col from string to datetime

  # pnl['Year'] = pnl['fiscalDateEnding'].dt.year # retrieve just the year from date col
  
  return pnl

In [39]:
def get_financial_statements_new(ticker):
  # new method to get all 3 financial statements
  # 
  # 
  # TODO: to refactorize this in future
  # we can also get EARNINGS (EPS), OVERVIEW (financial ratios), EARNINGS_CALENDAR 
  # (expected earnings in the next 3,6,12 months)

  pnl = pd.DataFrame()
  bs = pd.DataFrame()
  cf = pd.DataFrame()
  

  d = {'INCOME_STATEMENT':pnl,
       'BALANCE_SHEET':bs,
       'CASH_FLOW':cf}

  for fs in d.items():
    # INCOME_STATEMENT
    url = f'https://www.alphavantage.co/query?function=INCOME_STATEMENT&symbol={ticker}&apikey={API_KEY}'
    r = requests.get(url)
    data = r.json() # this returns a dict
    df = pd.DataFrame(data['annualReports'])  # retrieving the 'annualReports' value using its key from the dict
    df['fiscalDateEnding'] = pd.to_datetime(df['fiscalDateEnding']) # convert col from string to datetime
    df['Year'] = df['fiscalDateEnding'].dt.year # retrieve just the year from date col

    pnl = df

  return None

get_financial_statements_new(ticker)

pnl = pd.DataFrame()
bs = pd.DataFrame()
cf = pd.DataFrame()

d = {'INCOME_STATEMENT':pnl,
      'BALANCE_SHEET':bs,
      'CASH_FLOW':cf}

for fs in d.items():
  print(fs.key)
  print()

AttributeError: 'tuple' object has no attribute 'key'

In [75]:
def get_financial_statements(ticker):
    # new method to get all 3 financial statements
    # TODO: to refactorize this in future

    # d = {pnl:'INCOME_STATEMENT',
    #      bs:'BALANCE_SHEET',
    #      cf:'CASH_FLOW'}


    # INCOME_STATEMENT
    url = f'https://www.alphavantage.co/query?function=INCOME_STATEMENT&symbol={ticker}&apikey={API_KEY}'
    r = requests.get(url)
    data = r.json() # this returns a dict
    df = pd.DataFrame(data['annualReports'])  # retrieving the 'annualReports' value using its key from the dict
    df['fiscalDateEnding'] = pd.to_datetime(df['fiscalDateEnding']) # convert col from string to datetime
    df['Year'] = df['fiscalDateEnding'].dt.year # retrieve just the year from date col

    pnl = df

    # BALANCE_SHEET
    url = f'https://www.alphavantage.co/query?function=BALANCE_SHEET&symbol={ticker}&apikey={API_KEY}'
    r = requests.get(url)
    data = r.json() # this returns a dict
    df = pd.DataFrame(data['annualReports'])  
    df['fiscalDateEnding'] = pd.to_datetime(df['fiscalDateEnding']) 
    df['Year'] = df['fiscalDateEnding'].dt.year 

    bs = df

    # CASH_FLOW
    url = f'https://www.alphavantage.co/query?function=CASH_FLOW&symbol={ticker}&apikey={API_KEY}'
    r = requests.get(url)
    data = r.json() # this returns a dict
    df = pd.DataFrame(data['annualReports'])
    df['fiscalDateEnding'] = pd.to_datetime(df['fiscalDateEnding']) 
    df['Year'] = df['fiscalDateEnding'].dt.year 

    cf = df

    return pnl, bs, cf

In [126]:
pnl, bs, cf = get_financial_statements(ticker)

In [148]:
def merge_financial_statements(pnl, bs, cf):

    # TODO: maybe shouldn't merge the fin statements
    cf = cf.drop(columns=['netIncome']) # drop net income from cf as net income is in 2 financial statements

    pnl = pnl.set_index('Year')
    bs = bs.set_index('Year')
    cf = cf.set_index('Year')

    dataframes = [pnl, bs, cf]

    # merge
    df = pd.concat(dataframes, axis=1).reset_index()

    # 
    cols = ['operatingCashflow', 'capitalExpenditures', 'netIncome']

    for col in cols:
        df[col] = pd.to_numeric(df[col])

    # reverse the order (rows) of the dataframe
    df = df.iloc[::-1].reset_index(drop=True)

    return df

merged_fs = merge_financial_statements(pnl, bs, cf)

In [158]:
def project_future_revenues(df):
    '''
    Y1 & Y2 revenues are 'estimated' based on analysts report
    Y3 & Y4 revenues are 'projected' based on taking the min/avg revenue growth rate applied on current revenue
    '''

    ### Y1 and Y2 estimated revenues ###
    # Y1 and Y2 estimated revenue based on taking avg of analyst reports
    # TODO: double check if it's plus or minus
    df['freeCashFlowToEquity'] = df['operatingCashflow'] - df['capitalExpenditures']
    df['freeCashFlowRate'] = df['freeCashFlowToEquity'] / df['netIncome']

    # we'll use the avg FCF rather than the lowest
    fcf_rate = df['freeCashFlowRate'].mean()
    # fcf_rate = df['freeCashFlowRate'].min()

    # get_analyst_reports()


    ### Y3 and Y4 projected revenues ###
    df['totalRevenue'] = df['totalRevenue'].astype('int64')
    df['%_change'] = df['totalRevenue'].pct_change()
    
    revenue_growth_rate = df['%_change'].mean()
    print(revenue_growth_rate)

    return df

project_future_revenues(merged_fs)[['Year', 'totalRevenue', '%_change', 'netIncome']]



0.49003130057212224


,Year,totalRevenue,%_change,netIncome
0,2017,11759000000,NaN,-1962000000
1,2018,21461000000,0.825070,-976000000
2,2019,24578000000,0.145240,-862000000
3,2020,31536000000,0.283099,721000000
4,2021,53823000000,0.706716,5519000000


In [161]:
def project_future_income(df):

    df['netIncomeMargin'] = df['netIncome'] / df['totalRevenue']

    income_growth_rate = df['netIncomeMargin'].mean()
    print(income_growth_rate)

    return df

project_future_income(merged_fs)[['Year', 'netIncomeMargin']]

-0.02439964308571992


,Year,netIncomeMargin
0,2017,-0.166851
1,2018,-0.045478
2,2019,-0.035072
3,2020,0.022863
4,2021,0.102540


In [ ]:
def save_sp500_tickers():

    # desc: this returns a ... list? of all 500 ticker symbols for S&P500 companies

    df_companies = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
    tickers = df_companies[0]['Symbol']

    with open('sp500tickers.pickle', 'wb') as f:
    pickle.dump(tickers, f)

    return tickers

save_sp500_tickers()

In [ ]:
def get_data_from_yahoo(reload_sp500=False):
  
    if reload_sp500:
        tickers = save_sp500_tickers()
    else:
        with open('sp500tickers.pickle', 'rb') as f:
            tickers = pickle.load(f)

    return None